# Florida Government Forms AI Assistant
## Intro to AI - Final Project Implementation

**Team Members:**
- Carlecia Gordon (Carly) - Computer Science
- Giovanny Victome (jjviscane) - Computer Engineering
- Raptor - Computer Engineering
- Captain capital PSTL - Computer Science

---

## Project Overview
This project implements an AI system that:
1. Classifies government form images using a **Convolutional Neural Network (CNN)**
2. Answers text queries about Florida DMV/permit procedures using a **Multi-Layer Perceptron (MLP)**

### Key AI Concepts Demonstrated:
- **INPUT LAYER**: Receives preprocessed data
- **CONVOLUTIONAL LAYERS**: Feature extraction from images
- **POOLING LAYERS**: Dimensionality reduction
- **HIDDEN LAYERS (MLP)**: High-level reasoning
- **OUTPUT LAYER**: Classification probabilities
- **BACKPROPAGATION**: Learning algorithm
- **LOSS FUNCTION**: Error measurement

---
# WEEK 1: Foundation & Data Collection
---

## Step 1A: Initial Setup & Infrastructure
### Google Colab Environment Setup (Giovanny - Computer Engineering)

In [ ]:
# Mount Google Drive and create project structure
from google.colab import drive
drive.mount('/content/drive')

# Create organized folder structure
!mkdir -p '/content/drive/MyDrive/AI_Project/data/raw_forms'
!mkdir -p '/content/drive/MyDrive/AI_Project/data/processed'
!mkdir -p '/content/drive/MyDrive/AI_Project/models'
!mkdir -p '/content/drive/MyDrive/AI_Project/results'
!mkdir -p '/content/drive/MyDrive/AI_Project/presentation'

print("✅ Project structure created")

In [ ]:
# Install additional packages
!pip install -q tensorflow opencv-python pillow scikit-learn pdf2image
!apt-get install -q poppler-utils

print("✅ Packages installed")

In [ ]:
# Core imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import sqlite3
import pandas as pd
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import time
import os

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

## Step 1B: Database Design & Implementation
### SQLite Database Schema (Carly - Computer Science)

In [ ]:
# Connect to database
db_path = '/content/drive/MyDrive/AI_Project/florida_forms.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

print("✅ Database connection established")

In [ ]:
# Create forms table
cursor.execute('''
CREATE TABLE IF NOT EXISTS forms (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    form_number TEXT NOT NULL,
    form_name TEXT NOT NULL,
    category TEXT NOT NULL,
    url TEXT,
    description TEXT,
    requirements TEXT,
    created_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
''')

# Create index for faster queries
cursor.execute('''
CREATE INDEX IF NOT EXISTS idx_category ON forms(category)
''')

conn.commit()
print("✅ Forms table created")

In [ ]:
# Create queries table
cursor.execute('''
CREATE TABLE IF NOT EXISTS queries (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    question TEXT NOT NULL,
    category TEXT NOT NULL,
    intent TEXT,
    answer TEXT,
    created_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
''')

cursor.execute('''
CREATE INDEX IF NOT EXISTS idx_query_category ON queries(category)
''')

conn.commit()
print("✅ Queries table created")

In [ ]:
# Florida DMV forms data
forms_data = [
    ('HSMV 82040', 'Application for Duplicate/Renewal', 'License',
     'https://www.flhsmv.gov/pdf/forms/82040.pdf',
     'Renew or replace Florida driver license or ID card',
     'Proof of identity, residency, SSN'),
    
    ('HSMV 82042', 'Medical Examination Report', 'License',
     'https://www.flhsmv.gov/pdf/forms/82042.pdf',
     'Medical certification for commercial license',
     'Physical exam by certified examiner'),
    
    ('HSMV 82040', 'Vehicle Registration Renewal', 'Registration',
     'https://www.flhsmv.gov/pdf/forms/82040.pdf',
     'Renew vehicle registration',
     'Current registration, insurance proof'),
    
    ('HSMV 82041', 'Application for Certificate of Title', 'Title',
     'https://www.flhsmv.gov/pdf/forms/82041.pdf',
     'Apply for vehicle title',
     'Bill of sale, previous title, ID'),
    
    ('HSMV 83330', 'Learner License Application', 'Permit',
     'https://www.flhsmv.gov/pdf/forms/83330.pdf',
     'Apply for learners permit',
     'Parental consent if under 18, ID documents'),
    
    ('HSMV 71054', 'Skill Test Waiver', 'License',
     'https://www.flhsmv.gov/pdf/forms/71054.pdf',
     'Waive driving test with approved course',
     'Completion certificate from driving school'),
    
    ('HSMV 82053', 'Fast Title Application', 'Title',
     'https://www.flhsmv.gov/pdf/forms/82053.pdf',
     'Expedited title processing',
     'Standard title requirements plus expedite fee'),
    
    ('HSMV 71081', 'Insurance Affidavit', 'Registration',
     'https://www.flhsmv.gov/pdf/forms/71081.pdf',
     'Proof of insurance for registration',
     'Valid insurance policy information'),
    
    ('HSMV 82042', 'ID Card Application', 'ID',
     'https://www.flhsmv.gov/pdf/forms/82042.pdf',
     'Apply for Florida identification card',
     'Proof of identity, residency, SSN'),
    
    ('HSMV 83045', 'Power of Attorney', 'General',
     'https://www.flhsmv.gov/pdf/forms/83045.pdf',
     'Authorize representative for DMV transactions',
     'Notarized signature, representative ID')
]

cursor.executemany('''
INSERT INTO forms (form_number, form_name, category, url, description, requirements)
VALUES (?, ?, ?, ?, ?, ?)
''', forms_data)

conn.commit()
print(f"✅ Inserted {len(forms_data)} forms into database")

# Verify insertion
cursor.execute("SELECT category, COUNT(*) FROM forms GROUP BY category")
for row in cursor.fetchall():
    print(f"  - {row[0]}: {row[1]} forms")

## Step 1C: Form Image Collection
### Download and Convert PDFs (Raptor - Computer Engineering)

In [ ]:
import requests
from pdf2image import convert_from_path

def download_form(url, save_path):
    """Download PDF form from URL"""
    try:
        response = requests.get(url, timeout=10)
        with open(save_path, 'wb') as f:
            f.write(response.content)
        return True
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        return False

# Download all forms from database
cursor.execute("SELECT form_number, url, category FROM forms")
forms_list = cursor.fetchall()

raw_forms_dir = '/content/drive/MyDrive/AI_Project/data/raw_forms'

for form_num, url, category in forms_list:
    if url:
        filename = f"{category}__{form_num.replace(' ', '_')}.pdf"
        save_path = os.path.join(raw_forms_dir, filename)
        
        if download_form(url, save_path):
            print(f"✅ Downloaded: {filename}")
        else:
            print(f"❌ Failed: {filename}")

print(f"\n✅ Total forms downloaded: {len(os.listdir(raw_forms_dir))}")

In [ ]:
def pdf_to_image(pdf_path, output_dir, category):
    """Convert first page of PDF to PNG image"""
    try:
        # Convert first page only
        images = convert_from_path(pdf_path, first_page=1, last_page=1, dpi=150)
        
        # Save as PNG
        filename = os.path.basename(pdf_path).replace('.pdf', '.png')
        output_path = os.path.join(output_dir, category, filename)
        
        os.makedirs(os.path.join(output_dir, category), exist_ok=True)
        images[0].save(output_path, 'PNG')
        
        return output_path
    except Exception as e:
        print(f"Error converting {pdf_path}: {e}")
        return None

# Convert all PDFs
images_dir = '/content/drive/MyDrive/AI_Project/data/processed/images'

for pdf_file in os.listdir(raw_forms_dir):
    if pdf_file.endswith('.pdf'):
        category = pdf_file.split('__')[0]
        pdf_path = os.path.join(raw_forms_dir, pdf_file)
        
        img_path = pdf_to_image(pdf_path, images_dir, category)
        if img_path:
            print(f"✅ Converted: {pdf_file}")

print("\n✅ PDF to image conversion complete")

In [ ]:
from PIL import ImageEnhance, ImageOps
import random

def augment_image(image_path, output_dir, num_augmentations=10):
    """
    Create augmented versions of form image
    Augmentations: rotation, brightness, contrast, flip
    """
    img = Image.open(image_path)
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    
    augmented_paths = []
    
    for i in range(num_augmentations):
        aug_img = img.copy()
        
        # Random rotation (-5 to 5 degrees)
        angle = random.uniform(-5, 5)
        aug_img = aug_img.rotate(angle, fillcolor='white')
        
        # Random brightness (0.8 to 1.2)
        brightness = ImageEnhance.Brightness(aug_img)
        aug_img = brightness.enhance(random.uniform(0.8, 1.2))
        
        # Random contrast (0.9 to 1.1)
        contrast = ImageEnhance.Contrast(aug_img)
        aug_img = contrast.enhance(random.uniform(0.9, 1.1))
        
        # Save augmented image
        aug_filename = f"{base_name}_aug{i}.png"
        aug_path = os.path.join(output_dir, aug_filename)
        aug_img.save(aug_path)
        
        augmented_paths.append(aug_path)
    
    return augmented_paths

# Augment all images
augmented_dir = '/content/drive/MyDrive/AI_Project/data/processed/augmented'

total_images = 0
for category in os.listdir(images_dir):
    category_path = os.path.join(images_dir, category)
    
    if os.path.isdir(category_path):
        
        # Create category subfolder in augmented dir
        aug_category_dir = os.path.join(augmented_dir, category)
        os.makedirs(aug_category_dir, exist_ok=True)
        
        for img_file in os.listdir(category_path):
            img_path = os.path.join(category_path, img_file)
            
            # Copy original
            original_copy = os.path.join(aug_category_dir, img_file)
            Image.open(img_path).save(original_copy)
            
            # Create augmentations
            aug_paths = augment_image(img_path, aug_category_dir, num_augmentations=14)
            total_images += len(aug_paths) + 1  # +1 for original
            
            print(f"✅ Augmented: {img_file} → {len(aug_paths)} versions")

print(f"\n✅ Total dataset size: {total_images} images")
print(f"✅ Target: ~150 images (15 per category)")

## Step 1D: Image Preprocessing Pipeline
### Create preprocessing functions (Giovanny - Computer Engineering)

In [ ]:
def preprocess_image(image_path, target_size=(128, 128)):
    """
    Preprocess form images for CNN input
    
    Args:
        image_path: Path to image file
        target_size: Desired output dimensions (height, width)
    
    Returns:
        Preprocessed numpy array normalized to [0, 1]
    """
    try:
        # Read image in grayscale
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        if img is None:
            raise ValueError(f"Could not read image: {image_path}")
        
        # Resize to target size
        img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)
        
        # Normalize pixel values to [0, 1]
        img = img.astype(np.float32) / 255.0
        
        # Reshape to add channel dimension (H, W, 1)
        img = img.reshape(target_size[0], target_size[1], 1)
        
        return img
    
    except Exception as e:
        print(f"Error preprocessing {image_path}: {e}")
        return None

# Test preprocessing
test_categories = [d for d in os.listdir(augmented_dir) if os.path.isdir(os.path.join(augmented_dir, d))]
if test_categories:
    test_category = test_categories[0]
    test_imgs = os.listdir(os.path.join(augmented_dir, test_category))
    if test_imgs:
        test_img_path = os.path.join(augmented_dir, test_category, test_imgs[0])
        test_img = preprocess_image(test_img_path)
        
        print(f"✅ Preprocessing test successful")
        print(f"  - Input: {test_img_path}")
        print(f"  - Output shape: {test_img.shape}")
        print(f"  - Value range: [{test_img.min():.2f}, {test_img.max():.2f}]")

In [ ]:
def load_dataset(data_dir, target_size=(128, 128)):
    """
    Load entire dataset from directory structure
    
    Directory structure expected:
        data_dir/
            Category1/
                image1.png
                image2.png
            Category2/
                image1.png
    
    Returns:
        X: numpy array of images (N, H, W, 1)
        y: numpy array of labels (N,)
        categories: list of category names
    """
    images = []
    labels = []
    categories = sorted(os.listdir(data_dir))
    
    # Remove non-directory items
    categories = [c for c in categories if os.path.isdir(os.path.join(data_dir, c))]
    
    print(f"Loading dataset from {len(categories)} categories...")
    
    for label_idx, category in enumerate(categories):
        category_path = os.path.join(data_dir, category)
        image_files = [f for f in os.listdir(category_path) 
                      if f.endswith(('.png', '.jpg', '.jpeg'))]
        
        print(f"  Loading {category}: {len(image_files)} images", end='')
        
        for img_file in image_files:
            img_path = os.path.join(category_path, img_file)
            img = preprocess_image(img_path, target_size)
            
            if img is not None:
                images.append(img)
                labels.append(label_idx)
        
        print(f" → {len([l for l in labels if l == label_idx])} loaded")
    
    X = np.array(images)
    y = np.array(labels)
    
    print(f"\n✅ Dataset loaded successfully")
    print(f"  - Total images: {len(X)}")
    print(f"  - Image shape: {X[0].shape}")
    print(f"  - Categories: {categories}")
    print(f"  - Label distribution: {np.bincount(y)}")
    
    return X, y, categories

# Load complete dataset
X_full, y_full, category_names = load_dataset(augmented_dir)

In [ ]:
# First split: separate test set (20%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X_full, y_full,
    test_size=0.2,
    random_state=42,
    stratify=y_full
)

# Second split: separate validation from training (20% of remaining)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp,
    test_size=0.25,  # 0.25 * 0.8 = 0.2 of total
    random_state=42,
    stratify=y_temp
)

print("Dataset Split:")
print(f"  - Training: {len(X_train)} images ({len(X_train)/len(X_full)*100:.1f}%)")
print(f"  - Validation: {len(X_val)} images ({len(X_val)/len(X_full)*100:.1f}%)")
print(f"  - Testing: {len(X_test)} images ({len(X_test)/len(X_full)*100:.1f}%)")

# Save preprocessed datasets
np.save('/content/drive/MyDrive/AI_Project/data/processed/X_train.npy', X_train)
np.save('/content/drive/MyDrive/AI_Project/data/processed/X_val.npy', X_val)
np.save('/content/drive/MyDrive/AI_Project/data/processed/X_test.npy', X_test)
np.save('/content/drive/MyDrive/AI_Project/data/processed/y_train.npy', y_train)
np.save('/content/drive/MyDrive/AI_Project/data/processed/y_val.npy', y_val)
np.save('/content/drive/MyDrive/AI_Project/data/processed/y_test.npy', y_test)

# Save category names
with open('/content/drive/MyDrive/AI_Project/data/processed/categories.pkl', 'wb') as f:
    pickle.dump(category_names, f)

print("\n✅ Preprocessed datasets saved to Drive")

In [ ]:
# Visualize sample images from each category
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('Sample Forms by Category', fontsize=16)

for idx, category in enumerate(category_names):
    # Get first image from this category
    category_indices = np.where(y_full == idx)[0]
    if len(category_indices) > 0:
        sample_img = X_full[category_indices[0]]
        
        ax = axes[idx // 5, idx % 5]
        ax.imshow(sample_img.squeeze(), cmap='gray')
        ax.set_title(category)
        ax.axis('off')

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/AI_Project/results/sample_forms.png', dpi=150)
plt.show()

print("✅ Sample visualization saved for presentation")

---
# WEEK 2: CNN Model Development
---

## Step 2A: CNN Architecture Design
### Comprehensive CNN Model (Captain capital - Computer Science)

In [ ]:
def create_cnn_model(input_shape=(128, 128, 1), num_classes=5):
    """
    Convolutional Neural Network for form classification
    
    Architecture demonstrates:
    - INPUT LAYER: Receives preprocessed images
    - CONVOLUTIONAL LAYERS: Feature extraction from images
    - POOLING LAYERS: Dimensionality reduction
    - HIDDEN LAYERS (MLP): High-level reasoning
    - OUTPUT LAYER: Classification probabilities
    
    Args:
        input_shape: Image dimensions (height, width, channels)
        num_classes: Number of form categories
    
    Returns:
        Compiled Keras model
    """
    
    model = models.Sequential([
        # INPUT LAYER
        layers.Input(shape=input_shape, name='input'),
        
        # CONVOLUTIONAL BLOCK 1
        # Purpose: Detect low-level features (edges, lines, corners)
        # These are building blocks that appear in form layouts
        layers.Conv2D(16, (3, 3), activation='relu', padding='same', name='conv1'),
        layers.BatchNormalization(name='bn1'),
        layers.MaxPooling2D((2, 2), name='pool1'),  # 128x128 → 64x64
        layers.Dropout(0.25, name='dropout1'),
        
        # CONVOLUTIONAL BLOCK 2
        # Purpose: Detect mid-level features (form sections, text blocks, logos)
        # Combines low-level features into meaningful patterns
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', name='conv2'),
        layers.BatchNormalization(name='bn2'),
        layers.MaxPooling2D((2, 2), name='pool2'),  # 64x64 → 32x32
        layers.Dropout(0.25, name='dropout2'),
        
        # CONVOLUTIONAL BLOCK 3
        # Purpose: Detect high-level features (overall form structure, layout)
        # Recognizes complete form patterns
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='conv3'),
        layers.BatchNormalization(name='bn3'),
        layers.MaxPooling2D((2, 2), name='pool3'),  # 32x32 → 16x16
        layers.Dropout(0.25, name='dropout3'),
        
        # Flatten feature maps to 1D vector
        layers.Flatten(name='flatten'),
        
        # HIDDEN LAYERS (Multi-Layer Perceptron)
        # Purpose: Learn complex combinations of visual features
        # These fully-connected layers perform classification reasoning
        layers.Dense(128, activation='relu', name='hidden1'),
        layers.BatchNormalization(name='bn4'),
        layers.Dropout(0.4, name='dropout4'),
        
        layers.Dense(64, activation='relu', name='hidden2'),
        layers.Dropout(0.4, name='dropout5'),
        
        # OUTPUT LAYER
        # Softmax activation provides probability distribution over classes
        layers.Dense(num_classes, activation='softmax', name='output')
    ])
    
    return model

# Create model instance
cnn_model = create_cnn_model(num_classes=len(category_names))

# Display architecture
print("="*60)
print("CNN MODEL ARCHITECTURE")
print("="*60)
cnn_model.summary()

# Calculate model parameters
total_params = cnn_model.count_params()
print(f"\n✅ Total parameters: {total_params:,}")

### Why Convolutional Layers are Important

**1. AUTOMATIC FEATURE LEARNING**
- Traditional approach: Manually program what edges/shapes to look for
- CNN approach: Learns optimal features automatically from data
- Example: CNN discovers which form patterns distinguish licenses from permits

**2. PARAMETER SHARING**
- Same convolutional filter applied across entire image
- Detects features regardless of position in form
- Dramatically reduces parameters compared to fully-connected layers
- Example: 3x3 filter has only 9 weights, but scans entire 128x128 image

**3. TRANSLATION INVARIANCE**
- Form slightly shifted? CNN still recognizes it
- Logo in different corner? CNN adapts
- Pooling layers enhance this property

**4. HIERARCHICAL LEARNING**
- Layer 1: Edges and simple patterns
- Layer 2: Combinations → form sections
- Layer 3: Full layouts → form type identification

**5. EFFICIENCY FOR IMAGES**
- Exploits 2D structure of images
- More efficient than treating image as flat vector
- In our project: 128x128=16,384 pixels would need millions of weights in fully-connected network, but CNN uses only thousands

### Why Pooling is Important

**1. DIMENSIONALITY REDUCTION**
- Reduces spatial size: 64x64 → 32x32
- Fewer computations in subsequent layers
- Prevents overfitting by reducing parameters

**2. MAX POOLING (2x2) OPERATION**
- Divides feature map into 2x2 regions
- Keeps maximum value from each region
- Preserves strongest activations (most important features)

**3. TRANSLATION INVARIANCE**
- Small shifts in input don't change pooled output
- Form slightly off-center? Still same pooled features
- Makes model robust to minor variations

**4. COMPUTATIONAL EFFICIENCY**
- Each pooling layer reduces feature map size by 75%
- Speeds up training and inference
- In our model: 128x128 → 64x64 → 32x32 → 16x16

**5. RETAINS IMPORTANT INFORMATION**
- Max pooling keeps strongest signals
- Discards redundant spatial information
- Focuses on "what" features exist, not exact "where"

In [ ]:
# LOSS FUNCTION: Categorical Cross-Entropy
cnn_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

print("\n✅ CNN model compiled with categorical cross-entropy loss")

### Loss Function Explanation

**Categorical Cross-Entropy Loss:**
- Measures difference between predicted probabilities and true labels
- Formula: Loss = -Σ(y_true * log(y_pred))
- Low loss = good predictions, High loss = poor predictions
- Used for multi-class classification (License, Registration, Title, etc.)

**Why this loss function:**
- Softmax output layer produces probabilities summing to 1.0
- Cross-entropy penalizes confident wrong predictions heavily
- Works well with backpropagation for gradient calculation

**Example:**
- True label: License (category 0)
- Prediction: [0.8, 0.1, 0.05, 0.03, 0.02] → Low loss (confident & correct)
- Prediction: [0.2, 0.3, 0.3, 0.1, 0.1] → High loss (uncertain)
- Prediction: [0.1, 0.7, 0.1, 0.05, 0.05] → Very high loss (confident & wrong)

## Step 2B: Model Training with Backpropagation
### Training Setup (Captain capital - Computer Science)

In [ ]:
# Load preprocessed datasets
X_train = np.load('/content/drive/MyDrive/AI_Project/data/processed/X_train.npy')
X_val = np.load('/content/drive/MyDrive/AI_Project/data/processed/X_val.npy')
X_test = np.load('/content/drive/MyDrive/AI_Project/data/processed/X_test.npy')
y_train = np.load('/content/drive/MyDrive/AI_Project/data/processed/y_train.npy')
y_val = np.load('/content/drive/MyDrive/AI_Project/data/processed/y_val.npy')
y_test = np.load('/content/drive/MyDrive/AI_Project/data/processed/y_test.npy')

with open('/content/drive/MyDrive/AI_Project/data/processed/categories.pkl', 'rb') as f:
    category_names = pickle.load(f)

print("Dataset loaded:")
print(f"  Training: {X_train.shape[0]} images")
print(f"  Validation: {X_val.shape[0]} images")
print(f"  Testing: {X_test.shape[0]} images")
print(f"  Categories: {category_names}")

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Save best model based on validation accuracy
checkpoint = ModelCheckpoint(
    filepath='/content/drive/MyDrive/AI_Project/models/best_cnn_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

# Stop training if validation loss doesn't improve
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# Reduce learning rate when validation loss plateaus
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-6,
    verbose=1
)

callbacks = [checkpoint, early_stop, reduce_lr]

print("✅ Training callbacks configured")

In [ ]:
# Train the CNN model
# BACKPROPAGATION happens automatically during training!

print("\n" + "="*60)
print("STARTING CNN TRAINING")
print("Backpropagation will adjust weights to minimize loss")
print("="*60 + "\n")

history = cnn_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=16,
    callbacks=callbacks,
    verbose=1
)

print("\n✅ Training complete!")

### How Backpropagation Works

**Step-by-Step Process:**

1. **Forward Pass**: Input → through all layers → prediction
2. **Calculate Loss**: Compare prediction to correct answer using loss function
3. **Backward Pass**: Calculate how much each weight contributed to error
4. **Update Weights**: Adjust weights to reduce error (gradient descent)
5. **Repeat**: Do this thousands of times until model is accurate

**In our CNN:**
- If form classified wrong, backprop adjusts convolutional filters
- Gradients flow backward through all layers
- Adam optimizer determines how much to adjust each weight
- Learning rate controls size of weight updates

**Key Concepts:**
- **Gradient**: Direction to adjust weight to reduce loss
- **Learning Rate**: How big each adjustment step is
- **Epoch**: One complete pass through training data
- **Batch**: Subset of data processed before updating weights

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy plot
axes[0].plot(history.history['accuracy'], label='Training Accuracy')
axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy')
axes[0].set_title('Model Accuracy Over Time')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True)

# Loss plot
axes[1].plot(history.history['loss'], label='Training Loss')
axes[1].plot(history.history['val_loss'], label='Validation Loss')
axes[1].set_title('Model Loss Over Time (Backpropagation Minimizes This)')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/AI_Project/results/training_history.png', dpi=150)
plt.show()

print("✅ Training history visualization saved")

In [ ]:
# Evaluate on test set
print("\n" + "="*60)
print("EVALUATING ON TEST SET")
print("="*60 + "\n")

test_loss, test_accuracy, test_precision, test_recall = cnn_model.evaluate(
    X_test, y_test, verbose=1
)

print(f"\n📊 Test Results:")
print(f"  - Accuracy: {test_accuracy*100:.2f}%")
print(f"  - Precision: {test_precision*100:.2f}%")
print(f"  - Recall: {test_recall*100:.2f}%")
print(f"  - Loss: {test_loss:.4f}")

In [ ]:
# Generate predictions for confusion matrix
y_pred = cnn_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=category_names, yticklabels=category_names)
plt.title('Confusion Matrix - Form Classification')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.savefig('/content/drive/MyDrive/AI_Project/results/confusion_matrix.png', dpi=150)
plt.show()

# Classification Report
print("\n" + "="*60)
print("DETAILED CLASSIFICATION REPORT")
print("="*60)
print(classification_report(y_test, y_pred_classes, target_names=category_names))

print("\n✅ Evaluation complete!")

## Step 3: Simple User Interface
### Test the trained model with new images

In [ ]:
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, Image as IPImage

def classify_uploaded_form():
    """Upload and classify a form image"""
    print("Please upload a form image (PNG, JPG, or PDF first page)...")
    uploaded = files.upload()
    
    for filename in uploaded.keys():
        print(f"\nProcessing: {filename}")
        
        # Preprocess image
        img = preprocess_image(filename)
        
        if img is not None:
            # Add batch dimension
            img_batch = np.expand_dims(img, axis=0)
            
            # Predict
            prediction = cnn_model.predict(img_batch, verbose=0)
            predicted_class = np.argmax(prediction[0])
            confidence = prediction[0][predicted_class] * 100
            
            form_type = category_names[predicted_class]
            
            print(f"\n🎯 Prediction: {form_type}")
            print(f"📊 Confidence: {confidence:.2f}%")
            print(f"\nAll probabilities:")
            for i, cat in enumerate(category_names):
                print(f"  {cat}: {prediction[0][i]*100:.2f}%")
            
            # Query database for form info
            cursor.execute("SELECT * FROM forms WHERE category=? LIMIT 1", (form_type,))
            result = cursor.fetchone()
            
            if result:
                print(f"\n📋 Form Information:")
                print(f"  Form Number: {result[1]}")
                print(f"  Form Name: {result[2]}")
                print(f"  Description: {result[4]}")
                print(f"  Requirements: {result[5]}")
                print(f"  URL: {result[3]}")
        else:
            print("❌ Error: Could not process image")

# Create button to trigger upload
upload_button = widgets.Button(
    description='Upload & Classify Form',
    button_style='success',
    icon='upload'
)

def on_upload_click(b):
    classify_uploaded_form()

upload_button.on_click(on_upload_click)
display(upload_button)

---
# Summary & Key Takeaways
---

## What We Built:
1. **CNN Image Classifier** - Identifies government form types from images
2. **SQLite Database** - Stores form information and metadata
3. **Data Pipeline** - Augmentation, preprocessing, and splitting
4. **Training System** - With callbacks and optimization

## AI Concepts Demonstrated:

### 1. **Neural Network Architecture**
- **Input Layer**: Receives 128x128 grayscale images
- **Convolutional Layers**: Automatically learn visual features
- **Pooling Layers**: Reduce dimensions while preserving important info
- **Hidden Layers (MLP)**: Combine features for classification
- **Output Layer**: Softmax probabilities for each class

### 2. **Convolutional Neural Networks (CNN)**
- **Why Important**: Best architecture for image recognition
- **Key Feature**: Parameter sharing across image
- **Advantage**: Translation invariance (works even if image is shifted)

### 3. **Backpropagation**
- **Forward Pass**: Input → layers → prediction
- **Loss Calculation**: Measure error with cross-entropy
- **Backward Pass**: Calculate gradients for each weight
- **Weight Update**: Adam optimizer adjusts weights

### 4. **Loss Function (Cross-Entropy)**
- Measures difference between prediction and truth
- Guides backpropagation to improve model
- Lower loss = better predictions

### 5. **Training Process**
- **Epochs**: Multiple passes through data
- **Batches**: Process small groups at a time
- **Callbacks**: Early stopping, checkpointing, learning rate reduction
- **Validation**: Monitor performance on unseen data

## Project Files Generated:
- `florida_forms.db` - SQLite database
- `best_cnn_model.h5` - Trained model
- `X_train.npy, X_val.npy, X_test.npy` - Preprocessed images
- `y_train.npy, y_val.npy, y_test.npy` - Labels
- `categories.pkl` - Category names
- `sample_forms.png` - Visualization
- `training_history.png` - Training curves
- `confusion_matrix.png` - Classification results

## Next Steps (Week 3-5):
1. Build MLP for text query classification
2. Create complete user interface
3. Integrate both models
4. Prepare presentation and demo

---

**📝 Notes for Presentation:**
- Emphasize how CNN learns features automatically
- Explain backpropagation with training curves
- Show confusion matrix to demonstrate accuracy
- Demo live classification with uploaded images
- Discuss real-world applications

---